In [1]:
from torchrecsys.models.retrieval import DeepRetriever
from torchrecsys.external_datasets import Movielens_1M
from torchrecsys.datasets import InteractionsDataset
from pytorch_lightning import Trainer
from torchrecsys.models.scoring import NCF
import pandas as pd
import torch
import numpy as np
from torchrecsys.layers import retrieve_nearest_neighbors

In [2]:
data = Movielens_1M()
ratings, users, movies = data.load()

# Data processing

In [3]:
#Preprocess users
users['gender'], uniques = pd.factorize(users['gender'])
users['occupation'], uniques = pd.factorize(users['occupation'])
users['zip'], uniques = pd.factorize(users['zip'])
##Set category dtype
users['gender'] = users.gender.astype('category')
users['occupation'] = users.occupation.astype('category')
users['zip'] = users.zip.astype('category')

#Preprocess movies, 
##categories to index
movies['title'], uniques = pd.factorize(movies['title'])
movies['genres'], uniques = pd.factorize(movies['genres'])
##Set category dtype
movies['title'] = movies.title.astype('category')
movies['genres'] = movies.genres.astype('category')

#Make all ratings an implicit interaction
ratings["rating"] = ratings["rating"].apply(lambda x: 1 if x>=3 else 0)

In [4]:
dataset = InteractionsDataset(ratings, users, movies, item_id="movie_id", interaction_id="rating", sample_negatives=3)

# Retrieval Step

In [5]:
retriever = DeepRetriever(dataset.data_schema)

In [6]:
retriever.fit(dataset=dataset, num_epochs=1)

Epoch: 0/1, Loss: 24.83: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:34<00:00, 34.19s/it]


In [7]:
item_alias, item_representations = retriever.generate_item_representations(dataset)
user_alias, user_representations = retriever.generate_user_representations(dataset)

/Users/jiwidi/projects/github/torchrecsys/torchrecsys/models/retrieval/deep_retriever.py:159: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  items_features = torch.tensor(items_features)


In [8]:
nearest_neighbors_idx = retrieve_nearest_neighbors(candidates=item_representations, query=user_representations[0]).tolist()
retrieved_items = item_alias[nearest_neighbors_idx]
retrieved_items

tensor([  46, 3244, 3650, 1602, 3273, 3035, 3612, 1432, 1289, 1496,  376, 2983,
         805, 3714])

# Scoring step

In [9]:
scorer = NCF(dataset.data_schema)

In [10]:
scorer.fit(dataset=dataset, num_epochs=1)

Epoch: 0/1, Loss: 0.38: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:29<00:00, 29.72s/it]


In [11]:
user = user_alias[:10].tolist()
items = retrieved_items[:10].tolist()

user_features = dataset.get_user_features(user)
item_features = dataset.get_item_features(items)

pairs = torch.tensor([user, items]).T
user_features = torch.tensor(user_features)
items_features = torch.tensor(item_features)

scorer.predict(pairs, user_features, items_features)

tensor([[-1.0200],
        [-0.7300],
        [-4.9884],
        [-3.0428],
        [-0.4006],
        [ 0.0354],
        [-2.2178],
        [-1.5980],
        [-0.5185],
        [-2.1092]], grad_fn=<AddmmBackward0>)